<a href="https://colab.research.google.com/github/poietes5150/SuperCoding/blob/main/PyTorch_AutoGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch2 - PyTorch 모델 구성 요소 이해
PyTorch에서 신경망을 구성하는 핵심 클래스들인 `Autograde`, `nn.Module`, `Optimizer`, `Loss Function`, `DataLoader`에 대해 배워 봅니다.

*   Reference
    *   https://pytorch.org/docs/stable/tensors.html
    *   https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html
    *   https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html





# PyTorch Autograd 기능
PyTorch의 자동 미분 시스템인 **`autograd`**를 이해하고 실습해봅니다. 기울기 계산, 연산 그래프 추적, **`.backward()`, `.grad`**속성 등을 실제 코드와 함께 설명합니다.




Autograd란?

`autograd`는 PyTorch에서 **자동으로 기울기를 계산**해주는 엔진입니다. 수학적으로는 '연산 그래프(computational graph)'를 구성하고, `backward()`를 호출했을때 그래프를 따라 미분을 수행합니다.

PyTorch는 모든 텐서 연산을 추적하여, 이후` .backward()`를 호출하면 출력에 대한 입력의 미분 값을 `.grad` 속성에 자동으로 저장해줍니다.

# 예제1:스칼라 연산 미분

In [1]:
import torch

# 입력 텐서 (미분 추적 설정)
x = torch.tensor(2.0, requires_grad=True)

# 연산 수행
y = x**2 + 3 * x + 4

# 역전파
y.backward()

# 결과 출력
print(f"x의 값: {x.item()}")
print(f"y의 값: {y.item()}")
print(f"dy/dx = {x.grad}")  # 미분 결과
# print(f"x에 대한 y의 미분값: {x.grad}")

x의 값: 2.0
y의 값: 14.0
dy/dx = 7.0


설명

* `requires_grad=True`는 해당텐서가 미분 대상임을 의미합니다.
* `y.backward()` 는 y를 기준으로 역방향으로 그래프를 따라가며 미분을 수행합니다.
* `x.grad`에는 `dy/dx` 결과가 저장됩니다.
* 위 함수의 도함수는:     **dy/dx = 2x + 3 => 2 x 2 + 3 = 7**

---



# 예제 2: 벡터 연산 및 **`.backward()`** 사용

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2
y.backward()
print(x.grad) # what?

# 설명
* `.backward()`는 **스칼라 함수**에 대해서만 호출 가능한데 `y`는 벡터이므로 직접 `y.backward()`는 불가능합니다.
* autograd.backward()는 기본적으로 스칼라 출력에 대한 입력의 gradient(dy/dx)를 계산하는 함수입니다.
* 하지만 y가 벡터이면 각 요소마다 어떤 방향으로 미분할지 모호합니다.
* PyTorch는 이럴 때, 사용자에게 어떤 weight로 각 요소를 조합해서 스칼라로 만들지 알려달라고 요구합니다.
    * gradient vector (weights) 직접 제공하여 해결 가능
* 또는 `sum()`이나 `mean()`dmfh wnfdudi gkqslek.
    * z = 2x(1) + 2x(2) + 2x(3) 이므로 dz/dx = 2

In [2]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2

# y가 벡터일 때 backward()는 scalar여야 하므로 sum 사용
z = y.sum() # z = 2x1 + 2x2 + 2x3 = 12
z.backward()
print(x.grad) # 각각 2.0

tensor([2., 2., 2.])


In [7]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2

# y가 벡터일 때 backward()는 scalar여야 하므로 sum 사용
# 직접 gradient vector(weight) 제공
external_grad = torch.tensor([1.0, 1.0, 1.0])
y.backward(gradient=external_grad)
print(x.grad)

tensor([2., 2., 2.])


# 예제 3: z에 대한 w의 미분 값 dz/dw

In [8]:
# z에 대한 w의 미분값을 계산합니다.
w = torch.tensor(2.0, requires_grad=True)
y = w**2
z = 10*y + 50
z.backward()
print(f"dz/dw = {w.grad.item()}")

dz/dw = 40.0


# 예제 4: Q에대한 a와 b의 gradient dQ/da, dQ/db

In [9]:
# Q에 대한 a의 gradient
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 - b**2
external_grad = torch.tensor  ([1., 1.])
Q.backward(gradient=external_grad)
print(a.grad)

# Q에 대한 b의 gradient
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


# Autograd 비활성화: `requires_grad=False`

In [10]:
x = torch.tensor(3.0, requires_grad=False)
y = x * 2

print(f"y requires_grad? {y.requires_grad}")

y requires_grad? False


# Autograd 비활성화: `torch.no_grad()`

In [13]:
x = torch.tensor(3.0, requires_grad=True)
with torch.no_grad():
  y = x * 2
  print(f"y requires_grad? {y.requires_grad}") # False

y requires_grad? False


설명
* with torch.no_grad() 블록 안에서는 autograd가 작동하지 않습니다.
* 학습이 필요업는 예측 단계나 추론시 사용됩니다. (속도 개선)



# Autograd 연결 끊기:`.detach()`

In [15]:
x = torch.tensor(3.0, requires_grad=False)
y = x * 2
z = y.detach()

print(f"z requires_grad? {z.requires_grad}") # False

z requires_grad? False


설명
* `.detach()`는 autograd 연결이 끈긴 텐서를 반환합니다.
* 원본 텐서와는 값은 같지만, `.backward()`에 영향을 주지 않습니다.